In [1]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
from howlongtobeatpy import HowLongToBeat as hltb

In [2]:
list_of_games = pd.read_csv(r"C:\Users\ricar\Documents\Python Scripts\Game Backlog\game_list.csv", encoding='cp1252')

In [3]:
def extract_mc_data(series):
    url = 'https://www.metacritic.com/game/'
    name = series['name'].replace('.','').replace(',','').replace('-','').replace('\'','').replace(':','').replace(' ','-')
    session = requests.Session()
    session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    full_url = url + series['platform'] + '/' + name.lower() + '/'
    response = session.get(full_url)
    parser = BeautifulSoup(response.content, 'html.parser')
    
    review_count_elements = parser.find("span", class_="count")
    if review_count_elements is not None:
        review_count_text = review_count_elements.text.strip()
        review_count = int(re.findall(r'[\d]+', review_count_text)[0])
    else:
        review_count = 0
    
    score_elements = parser.find("span", itemprop="ratingValue")
    if score_elements is not None:
        score_text = score_elements.text.strip()
        score = int(score_text)
    else:
        score = 0
    return pd.Series([series['name'], series['platform'], score, review_count], index =['name', 'platform', 'score', 'total_reviews'])

In [4]:
print(extract_mc_data(list_of_games.iloc[34]))

name             Enter the Gungeon
platform             playstation-4
score                           82
total_reviews                   33
dtype: object


In [5]:
subset_10 = list_of_games.head(10)

In [6]:
subset_10 = subset_10.apply(extract_mc_data, axis='columns')

In [7]:
print(subset_10)

                              name       platform  score  total_reviews
0                             Abzu  playstation-4     78             72
1                  Alien Isolation  playstation-3      0              0
2  Animal Crossing Amiibo Festival          wii-u     46             20
3     Animal Crossing New Horizons         switch     90            111
4                 Assassin's Creed             pc      0              0
5                        Bayonetta          wii-u     86             18
6                      Bayonetta 2          wii-u     91             80
7                    Balloon Fight            NES      0              0
8             Batman Arkham Asylum  playstation-3     91             70
9                       BloodBorne  playstation-4     92            100


In [8]:
list_of_games = list_of_games.apply(extract_mc_data, axis='columns')

In [9]:
list_of_games.head(10)

,name,platform,score,total_reviews
0,Abzu,playstation-4,78,72
1,Alien Isolation,playstation-3,0,0
2,Animal Crossing Amiibo Festival,wii-u,46,20
3,Animal Crossing New Horizons,switch,90,111
4,Assassin's Creed,pc,0,0
5,Bayonetta,wii-u,86,18
6,Bayonetta 2,wii-u,91,80
7,Balloon Fight,NES,0,0
8,Batman Arkham Asylum,playstation-3,91,70
9,BloodBorne,playstation-4,92,100


In [10]:
list_of_games.tail(10)

,name,platform,score,total_reviews
516,Burnout Paradise: The Ultimate Box,pc,87,26
517,Peggle,pc,0,0
518,Bejeweled 3,pc,82,27
519,Dragon Age: Origins,pc,91,67
520,Syndicate,pc,69,16
521,Mass Effect 2,pc,94,55
522,Medal of Honor Pacific Assault,pc,80,43
523,SteamWorld Dig,pc,76,12
524,Plants vs. Zombies Game of the Year Edition,pc,0,0
525,The Sims 4,pc,70,75


In [11]:
def extract_hltb_data(name):
    matching_games = hltb().search(name)
    gametime = [0, 0, 0]
    if matching_games is not None:
        for game in matching_games:
            if name == game.game_name:
                gametime[0] = game.main_story
                gametime[1] = game.main_extra
                
    gametime[2] = (gametime[0] + gametime[1]) / 2
    return pd.Series(gametime)

In [12]:
print(extract_hltb_data(list_of_games.iloc[216]['name']))

0    10.84
1    10.94
2    10.89
dtype: float64


In [13]:
list_of_games[['main_story', 'main_extra', 'avg_playthrough']] = list_of_games['name'].apply(extract_hltb_data)

In [14]:
list_of_games.tail(10)

,name,platform,score,total_reviews,main_story,main_extra,avg_playthrough
516,Burnout Paradise: The Ultimate Box,pc,87,26,0.00,0.00,0.000
517,Peggle,pc,0,0,0.00,0.00,0.000
518,Bejeweled 3,pc,82,27,4.23,11.27,7.750
519,Dragon Age: Origins,pc,91,67,40.35,58.42,49.385
520,Syndicate,pc,69,16,12.05,12.22,12.135
521,Mass Effect 2,pc,94,55,24.42,35.47,29.945
522,Medal of Honor Pacific Assault,pc,80,43,0.00,0.00,0.000
523,SteamWorld Dig,pc,76,12,5.06,6.52,5.790
524,Plants vs. Zombies Game of the Year Edition,pc,0,0,0.00,0.00,0.000
525,The Sims 4,pc,70,75,18.16,68.40,43.280


In [15]:
list_of_games['normalized_score'] = list_of_games['score'].apply(
    lambda x: (x - list_of_games['score'].min())/(list_of_games['score'].max() - list_of_games['score'].min()))

In [16]:
list_of_games['normalized_reviews'] = list_of_games['total_reviews'].apply(
    lambda x: (x - list_of_games['total_reviews'].min())/(list_of_games['total_reviews'].max() - list_of_games['total_reviews'].min()))

In [17]:
list_of_games['normalized_time'] = list_of_games['avg_playthrough'].apply(
    lambda x: (x - list_of_games['avg_playthrough'].min())/(list_of_games['avg_playthrough'].max() - list_of_games['avg_playthrough'].min()))

In [18]:
list_of_games['playability_score'] = list_of_games.apply(
    lambda x: ((x['normalized_score']*0.5)+(x['normalized_reviews']*0.3)+(x['normalized_time']*0.2)) * 100,
    axis='columns')

In [19]:
list_of_games.sort_values(['playability_score'], inplace=True, ascending=False)
list_of_games[['name', 'platform', 'playability_score']].head(15)

,name,platform,playability_score
172,Super Mario Odyssey,switch,78.464728
184,The Last Of Us Part II,playstation-4,77.938144
53,God Of War,playstation-4,77.709807
187,The Legend of Zelda Breath of the Wild,switch,77.024793
207,Uncharted 4: A Thief's End,playstation-4,76.325600
203,Uncharted 2: Among Thieves,playstation-3,75.776869
234,Dota 2,pc,74.573571
3,Animal Crossing New Horizons,switch,73.912414
183,The Last Of Us,playstation-3,73.266593
9,BloodBorne,playstation-4,72.216069


In [20]:
list_of_games.to_csv(r"C:\Users\ricar\Documents\Python Scripts\Game Backlog\backlog.csv")